In [1]:
import os, sys
sys.path.append(os.path.abspath(os.path.join('../..'))) # access sibling directories
sys.path.append("C:\\Users\\lasse\\Documents\\GitHub\\BayesFlow")

from src.python.settings import summary_meta_diffusion, evidence_meta_diffusion
from src.python.helpers import load_simulated_rt_data, mask_inputs
from src.python.networks import HierarchicalInvariantNetwork, EvidentialNetwork

import numpy as np
from tensorflow.keras.experimental import CosineDecayRestarts
from tensorflow.keras.optimizers import Adam
from functools import partial

from bayesflow.trainers import ModelComparisonTrainer
from bayesflow.amortizers import MultiModelAmortizer 
from bayesflow.losses import log_loss

# Pre-training with 100 trials per person

## Load in data

In [2]:
levy_sims_folder = 'c:\\Users\\lasse\\documents\\hierarchical model comparison project\\data\\Levy_flight_application'

indices_100_filename = "train_indices_100_trials.npy"
datasets_100_filename = "train_datasets_100_trials.npy"

indices_100, datasets_100 = load_simulated_rt_data(levy_sims_folder, indices_100_filename, datasets_100_filename)

## Training

When retraining: Move to new trial folder so that networks are initialized from scratch.

In [4]:
summary_net = HierarchicalInvariantNetwork(summary_meta_diffusion)
evidence_net = EvidentialNetwork(evidence_meta_diffusion)
amortizer = MultiModelAmortizer(evidence_net, summary_net)

In [5]:
# Cosine Decay with Restarts
initial_lr = 0.0005
first_decay_steps = 1000
t_mul = 2
m_mul = 0.9
alpha = 0.2
lr_schedule_restart = CosineDecayRestarts(
    initial_lr, first_decay_steps, t_mul=t_mul, m_mul=m_mul, alpha=alpha
    )

# Checkpoint path for storing pretrained network
trial_folder = 'trial_2'
checkpoint_path = os.path.join(os.getcwd(), 'checkpoints', trial_folder)

trainer = ModelComparisonTrainer(
    network=amortizer, 
    loss=partial(log_loss, kl_weight=0),
    optimizer=partial(Adam, lr_schedule_restart),
    checkpoint_path=checkpoint_path,
    skip_checks=True
    )

TRAINER INITIALIZATION: No generative model provided. Only offline learning mode is available!
Initializing networks from scratch.


In [6]:
losses = trainer.train_offline(epochs=32, batch_size=32, 
                               model_indices=indices_100, sim_data=datasets_100)

Converting 40000 simulations to a TensorFlow data set...


Training epoch 1:   0%|          | 0/1250 [00:00<?, ?it/s]

Training epoch 2:   0%|          | 0/1250 [00:00<?, ?it/s]

Training epoch 3:   0%|          | 0/1250 [00:00<?, ?it/s]

Training epoch 4:   0%|          | 0/1250 [00:00<?, ?it/s]

...

Training epoch 28:   0%|          | 0/1250 [00:00<?, ?it/s]

Training epoch 29:   0%|          | 0/1250 [00:00<?, ?it/s]

Training epoch 30:   0%|          | 0/1250 [00:00<?, ?it/s]

Training epoch 31:   0%|          | 0/1250 [00:00<?, ?it/s]

Training epoch 32:   0%|          | 0/1250 [00:00<?, ?it/s]

results for trial 1 / trial 2
- training time = 62 minutes / 64 minutes
- running loss after 32 epochs = 0.469 / 0.462